In [30]:
# Dependencies
import pandas as pd

In [31]:
# Import files using pandas
items_df = pd.read_csv("generated_data\items_complete.csv")
players_df = pd.read_csv("generated_data\players_complete.csv")
purchase_df = pd.read_csv("generated_data\purchase_data_3.csv")

In [32]:
# Get the total number of players with .count(), using SN column b/c its short
player_count = players_df["SN"].count()

In [33]:
# Get the total count of unique items with len() since .unique() returns a list
unique_item_count = len(items_df["Item Name"].unique())

In [34]:
# Get the average purchase price by dividing the sum of the price column by total number of items
average_purchase_price = purchase_df["Price"].sum() / len(purchase_df["Price"])

In [42]:
# Get the total number of purchases
total_purchases = purchase_df["Price"].count()

78


In [57]:
# Get total revenue by taking price from purchase and subtracting price from item
# join table -> do more stuff
#purchase_items_joined_df = pd.merge(purchase_df, items_df, on="Item ID")
#purchase_items_joined_df = purchase_items_joined_df.rename(columns={"Item Name_x": "Item Name(Purchase)",
#                                                                   "Price_x": "Purchase Price",
#                                                                   "Item Name_y": "Item Name(Stock)",
#                                                                   "Price_y": "MSRP"})
#purchase_items_joined_df["Revenue"] = purchase_items_joined_df["Purchase Price"] - purchase_items_joined_df["MSRP"]
#purchase_items_joined_df.head() FFFFFFFF I SHOULD'VE CHECKED THE GENERATOR CODE TO SEE IF THE PRICE WAS DIFFERENT
# OK actually just add up the revenue from the purchase_df
total_revenue = purchase_df["Price"].sum()

228.0999999999999


In [45]:
purchase_df.head(1)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Iloni35,20,Male,93,Apocalyptic Battlescythe,4.49


In [44]:
items_df.head(1)

,Item ID,Item Name,Price
0,0,Splinter,1.89


In [ ]:
# Things to do
#PLAYER COUNT
# - Total Number of Players
#PURCHASING ANALYSIS(TOTAL)
# - Number of Unique Items
# - Average Purchase Price
# - Total Number of Purchases
# - Total Revenue
#GENDER DEMOGRAPHICS
#PURCHASING ANALYSIS (GENDER)
#AGE DEMOGRAPHICS
#TOP SPENDERS
#MOST POPULAR ITEMS
#MOST PROFITABLE ITEMS